In [12]:
#meg source localization
import numpy as np

import os.path as op

import pandas as pd

import mne
from mne import setup_volume_source_space
from mne.beamformer import apply_lcmv_cov, make_lcmv
from mne.coreg import Coregistration
from mne.cov import compute_covariance
from mne.datasets import somato

from pathlib import Path

import ipywidgets

%matplotlib qt

In [14]:
subjects = {#"Patient03" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/meg/FM_patient3/FM_9to16_part1_raw_tsss.fif",
            'patient_05_part1L' : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/meg/WZ_patient5/WZ_LHippo_001_tsss.fif" ,
            'patient_05_part1R' : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/meg/WZ_patient5/WZ_Rhippo_raw_001_tsss.fif",
            'patient_05_part2L' : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/meg/WZ_patient5/WZ_Lhippo_raw_002_tsss.fif",
            'patient_05_part2R' : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/meg/WZ_patient5/WZ_RHippo_raw_002_tsss.fif",
            "patient_06" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/meg/ZM_patient6/ZM_raw_001_tsss.fif",
            "patient_08" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/meg/XX_patient8/XX_raw_001_tsss.fif",
           # "patient_09" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/meg/TY_patient9/TY_raw_001_tsss.fif",
           "patient_10" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/meg/NY_patient10/NY_raw_001_tsss.fif",
          "patient_11_part1" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/meg/ZJ_patient11/ZJ_raw_001_tsss.fif",
            "patient_11_part2" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/meg/ZJ_patient11/ZJ_raw_002_tsss.fif",
            #"patient_12" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/meg/JC_patient12/JC_raw_001_tsss.fif",
            "patient_13" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/meg/ZB_patient13/ZB_raw_004_tsss.fif",
            "patient_14_part1_left" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/meg/CL_patient14/CL_raw_001_tsss.fif",
           "patient_14_part2_left" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/meg/CL_patient14/CL_raw_003_tsss.fif",
            "patient_14_part1_right" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/meg/CL_patient14/CL_raw_001_tsss.fif",
            "patient_14_part2_right" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/meg/CL_patient14/CL_raw_003_tsss.fif",
            #"patient_15" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/meg/ZX_patient15/ZX_EP_rest_001_tsss.fif",
            "patient_16" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/meg/RB_patient16/RB_EP_SEEG_rest_001_tsss.fif",
            "patient_17" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/meg/LJM_patient17/LJM_EP_SEEG_rest_001_tsss.fif",
            "patient_21" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/meg/HH_patient21/HH_SEEG_001_tsss.fif",
            "patient_22" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/meg/YY_patient22/YY_SEEG_raw_001_tsss.fif",
            "patient_23" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/meg/ZQ_patient23/ZQ_EP_SEEG_001_tsss.fif",
            "patient_25":  "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/meg/WL_patient25/WL_raw_rest_001_tsss.fif"
            }


In [7]:
#make a dictionary of coreg files
coregs_files = {"patient_03" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/coregs/patient_03-trans.fif",
            "patient_05_part1L" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/coregs/patient_05-trans.fif" ,
            "patient_05_part1R" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/coregs/patient_05-trans.fif" ,
            "patient_05_part2L" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/coregs/patient_05-trans.fif" ,
            "patient_05_part2R" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/coregs/patient_05-trans.fif" ,
            "patient_06" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/coregs/patient_06-trans.fif",
            "patient_08" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/coregs/patient_08-trans.fif",
            "patient_09" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/coregs/patient_09-trans.fif",
            "patient_10" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/coregs/patient_10-trans.fif",
            "patient_11_part1" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/coregs/patient_11-trans.fif",
            "patient_11_part2" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/coregs/patient_11-trans.fif",
            "patient_13" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/coregs/patient_13-trans.fif",
            "patient_14_part1_left" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/coregs/patient_14-trans.fif",
            "patient_14_part2_left" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/coregs/patient_14-trans.fif",
            "patient_14_part1_right" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/coregs/patient_14-trans.fif",
            "patient_14_part2_right" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/coregs/patient_14-trans.fif",
            "patient_16" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/coregs/patient_16-trans.fif",
            "patient_17" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/coregs/patient_17-trans.fif",
            "patient_21" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/coregs/patient_21-trans.fif",
            "patient_22" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/coregs/patient_22-trans.fif",
            "patient_23" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/coregs/patient_23-trans.fif",
            "patient_25":  "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/coregs/patient_25-trans.fif"
            }

In [15]:
#dictionary for epochsa
all_epochs_files = {#"patient_03" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/MEG_Epochs/Patient3_part1_all_SWR_epochs.fi",
            "patient_05_part1L" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/MEG_Epochs/theta/Patient5_part1L_all_theta_epochs.fif" ,
            "patient_05_part1R" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/MEG_Epochs/theta/Patient5_part1R_all_theta_epochs.fif" ,
            "patient_05_part2L" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/MEG_Epochs/theta/Patient5_part2L_all_theta_epochs.fif" ,
            "patient_05_part2R" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/MEG_Epochs/theta/Patient5_part2R_all_theta_epochs.fif" ,
            "patient_06" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/MEG_Epochs/theta/Patient6_all_theta_epochs.fif",
            "patient_08" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/MEG_Epochs/theta/Patient8_all_theta_epochs.fif",
            #"patient_09" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/MEG_Epochs/Patient9_all_SWR_epochs.fif",
            "patient_10" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/MEG_Epochs/theta/Patient10_all_theta_epochs.fif",
            "patient_11_part1" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/MEG_Epochs/theta/Patient11_part1_all_theta_epochs.fif",
            "patient_11_part2" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/MEG_Epochs/theta/Patient11_part2_all_theta_epochs.fif",
            "patient_13" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/MEG_Epochs/theta/Patient13_all_theta_epochs.fif",
            "patient_14_part1_left" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/MEG_Epochs/theta/Patient14_part1_left_all_theta_epochs.fif",
            "patient_14_part2_left" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/MEG_Epochs/theta/Patient14_part2_left_all_theta_epochs.fif",
            "patient_14_part1_right" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/MEG_Epochs/theta/Patient14_part1_right_all_theta_epochs.fif",
            "patient_14_part2_right" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/MEG_Epochs/theta/Patient14_part2_right_all_theta_epochs.fif",
            "patient_16" : '/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/MEG_Epochs/theta/Patient16_all_theta_epochs.fif',
            "patient_17" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/MEG_Epochs/theta/Patient17_all_theta_epochs.fif",
            "patient_21" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/MEG_Epochs/theta/Patient21_all_theta_epochs.fif",
            "patient_22" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/MEG_Epochs/theta/Patient22_all_theta_epochs.fif",
            "patient_23" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/MEG_Epochs/theta/Patient23_all_theta_epochs.fif",
            "patient_25":  "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/MEG_Epochs/theta/Patient25_all_theta_epochs.fif"
            }


high_epochs_files = {#"patient_03" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/MEG_Epochs/Patient3_part1_high_SWR_epochs.fi",
            "patient_05_part1L" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/MEG_Epochs/theta/Patient5_part1L_high_theta_epochs.fif" ,
            "patient_05_part1R" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/MEG_Epochs/theta/Patient5_part1R_high_theta_epochs.fif" ,
            "patient_05_part2L" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/MEG_Epochs/theta/Patient5_part2L_high_theta_epochs.fif" ,
            "patient_05_part2R" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/MEG_Epochs/theta/Patient5_part2R_high_theta_epochs.fif" ,
            "patient_06" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/MEG_Epochs/theta/Patient6_high_theta_epochs.fif",
            "patient_08" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/MEG_Epochs/theta/Patient8_high_theta_epochs.fif",
            #"patient_09" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/MEG_Epochs/Patient9_high_SWR_epochs.fif",
            "patient_10" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/MEG_Epochs/theta/Patient10_high_theta_epochs.fif",
            "patient_11_part1" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/MEG_Epochs/theta/Patient11_part1_high_theta_epochs.fif",
            "patient_11_part2" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/MEG_Epochs/theta/Patient11_part2_high_theta_epochs.fif",
            "patient_13" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/MEG_Epochs/theta/Patient13_high_theta_epochs.fif",
            "patient_14_part1_left" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/MEG_Epochs/theta/Patient14_part1_left_high_theta_epochs.fif",
            "patient_14_part2_left" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/MEG_Epochs/theta/Patient14_part2_left_high_theta_epochs.fif",
            "patient_14_part1_right" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/MEG_Epochs/theta/Patient14_part1_right_high_theta_epochs.fif",
            "patient_14_part2_right" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/MEG_Epochs/theta/Patient14_part2_right_high_theta_epochs.fif",
            "patient_16" : '/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/MEG_Epochs/theta/Patient16_high_theta_epochs.fif',
            "patient_17" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/MEG_Epochs/theta/Patient17_high_theta_epochs.fif",
            "patient_21" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/MEG_Epochs/theta/Patient21_high_theta_epochs.fif",
            "patient_22" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/MEG_Epochs/theta/Patient22_high_theta_epochs.fif",
            "patient_23" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/MEG_Epochs/theta/Patient23_high_theta_epochs.fif",
            "patient_25":  "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/MEG_Epochs/theta/Patient25_high_theta_epochs.fif"
            }

low_epochs_files = {#"patient_03" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/MEG_Epochs/Patient3_part1_low_SWR_epochs.fi",
            "patient_05_part1L" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/MEG_Epochs/theta/Patient5_part1L_low_theta_epochs.fif" ,
            "patient_05_part1R" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/MEG_Epochs/theta/Patient5_part1R_low_theta_epochs.fif" ,
            "patient_05_part2L" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/MEG_Epochs/theta/Patient5_part2L_low_theta_epochs.fif" ,
            "patient_05_part2R" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/MEG_Epochs/theta/Patient5_part2R_low_theta_epochs.fif" ,
            "patient_06" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/MEG_Epochs/theta/Patient6_low_theta_epochs.fif",
            "patient_08" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/MEG_Epochs/theta/Patient8_low_theta_epochs.fif",
            "patient_09" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/MEG_Epochs/theta/Patient9_low_theta_epochs.fif",
            "patient_10" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/MEG_Epochs/theta/Patient10_low_theta_epochs.fif",
            "patient_11_part1" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/MEG_Epochs/theta/Patient11_part1_low_theta_epochs.fif",
            "patient_11_part2" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/MEG_Epochs/theta/Patient11_part2_low_theta_epochs.fif",
            "patient_13" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/MEG_Epochs/theta/Patient13_low_theta_epochs.fif",
            "patient_14_part1_left" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/MEG_Epochs/theta/Patient14_part1_left_low_theta_epochs.fif",
            "patient_14_part2_left" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/MEG_Epochs/theta/Patient14_part2_left_low_theta_epochs.fif",
            "patient_14_part1_right" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/MEG_Epochs/theta/Patient14_part1_right_low_theta_epochs.fif",
            "patient_14_part2_right" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/MEG_Epochs/theta/Patient14_part2_right_low_theta_epochs.fif",
            "patient_16" : '/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/MEG_Epochs/theta/Patient16_low_theta_epochs.fif',
            "patient_17" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/MEG_Epochs/theta/Patient17_low_theta_epochs.fif",
            "patient_21" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/MEG_Epochs/theta/Patient21_low_theta_epochs.fif",
            "patient_22" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/MEG_Epochs/theta/Patient22_low_theta_epochs.fif",
            "patient_23" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/MEG_Epochs/theta/Patient23_low_theta_epochs.fif",
            "patient_25":  "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/MEG_Epochs/theta/Patient25_low_theta_epochs.fif"
            }

In [16]:
#probs have to improt raw and coreg files and eepochs
for subject, path in subjects.items():
    raw_fname = op.abspath( subjects[subject])
    raw = mne.io.read_raw_fif(raw_fname)
    info = raw.info

    base_subject = subject.split("_part")[0] 
    print(base_subject)
  
    
    data_path = op.abspath("/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS")
    subjects_dir = op.join(data_path, "freesurfer", "recons")
    coreg_fname = op.abspath(coregs_files[subject])
    coreg = mne.read_trans(coreg_fname)

    #improt 3 sets of epochs
    all_epochs_fname = op.abspath(all_epochs_files[subject])
    all_epochs = mne.read_epochs(all_epochs_fname)

    high_epochs_fname = op.abspath(high_epochs_files[subject])
    high_epochs = mne.read_epochs(high_epochs_fname)

    low_epochs_fname = op.abspath(low_epochs_files[subject])
    low_epochs = mne.read_epochs(low_epochs_fname)


    #filter in the ripple band
    all_epochs = all_epochs.filter(2, 5)
    high_epochs = high_epochs.filter(2, 5)
    low_epochs = low_epochs.filter(2, 5)

    #prepare source space
    surface = op.join(subjects_dir, base_subject , "bem" , "inner_skull.surf")
    src = mne.setup_volume_source_space(
    base_subject, subjects_dir=subjects_dir, pos=10.0, surface=surface, add_interpolator=False
    )  

    conductivity = (0.3,)
    model = mne.make_bem_model(
    subject=base_subject, ico=4, conductivity=conductivity, subjects_dir=subjects_dir
    )
    bem = mne.make_bem_solution(model)

    #compute forward solution 
    fwd = mne.make_forward_solution(
        raw_fname,
        trans=coreg,
        src=src,
        bem= bem,
        meg=True,
        eeg=False,
        mindist=5.0,
        n_jobs=None,
        verbose=True,
        )
   #prepare covariance matrix
    rank = mne.compute_rank(all_epochs, tol=1e-6, tol_kind="relative")
    active_win = (-0.1, 0.1)
    baseline_win = (-0.1, 0.1)
    common_win = (-0.1, 0.1)

    #epochs of no SWRs
    baseline_cov = compute_covariance(
        low_epochs,
        tmin=baseline_win[0],
        tmax=baseline_win[1],
        method="shrunk",
        rank=rank,
        verbose=True,
    )
    #epochs of SWRs
    active_cov = compute_covariance(
        high_epochs,
        tmin=active_win[0],
        tmax=active_win[1],
        method="shrunk",
        rank=rank,
        verbose=True,
    )

    #epochs of both
    cov = compute_covariance(
        all_epochs,
        tmin=common_win[0],
        tmax=common_win[1],
        method="shrunk",
        rank=rank,
        verbose=True,
    )

    noise_cov = mne.make_ad_hoc_cov(all_epochs.info) 

    # Weighted averaging is already in the addition of covariance objects.
    cov.plot(all_epochs.info)

    #calculate beamformer weights

    filters = make_lcmv(
            all_epochs.info, fwd, cov,
            reg=0.05,noise_cov=noise_cov,
            pick_ori="max-power", 
            weight_norm= None
        )


    stc_base = apply_lcmv_cov(baseline_cov, filters)
    stc_act = apply_lcmv_cov(active_cov, filters)


    stc_act.data = stc_act.transform_data(np.log10)
    stc_base.data = stc_base.transform_data(np.log10)

    stc_dB = stc_act - stc_base

    #stc_dB.plot(subject=base_subject,subjects_dir=subjects_dir,src=src,mode='glass_brain')

    #save 
    stc_dB.save_as_volume(f'/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/source_loc/theta/theta_{subject}.nii',src,  overwrite=True)
    #also save a stc file
    stc_dB.save(f'/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/stc_files/theta/theta_{subject}', overwrite = True)

Opening raw data file /Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/meg/WZ_patient5/WZ_LHippo_001_tsss.fif...
    Range : 289000 ... 646999 =    289.000 ...   646.999 secs
Ready.
patient_05
Reading /Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/MEG_Epochs/theta/Patient5_part1L_all_theta_epochs.fif ...
    Found the data of interest:
        t =   -3000.00 ...    3000.00 ms
        0 CTF compensation matrices available


/var/folders/hp/k3clndsn2sj608qsb4fl5zvr0000gn/T/ipykernel_57370/867120382.py:4: RuntimeWarning: This filename (/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/meg/WZ_patient5/WZ_LHippo_001_tsss.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw = mne.io.read_raw_fif(raw_fname)
/var/folders/hp/k3clndsn2sj608qsb4fl5zvr0000gn/T/ipykernel_57370/867120382.py:18: RuntimeWarning: This filename (/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/MEG_Epochs/theta/Patient5_part1L_all_theta_epochs.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  all_epochs = mne.read_epochs(all_epochs_fname)


Not setting metadata
59 matching events found
No baseline correction applied
0 projection items activated
Reading /Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/MEG_Epochs/theta/Patient5_part1L_high_theta_epochs.fif ...
    Found the data of interest:
        t =   -3000.00 ...    3000.00 ms
        0 CTF compensation matrices available


/var/folders/hp/k3clndsn2sj608qsb4fl5zvr0000gn/T/ipykernel_57370/867120382.py:18: RuntimeWarning: The events passed to the Epochs constructor are not chronologically ordered.
  all_epochs = mne.read_epochs(all_epochs_fname)
/var/folders/hp/k3clndsn2sj608qsb4fl5zvr0000gn/T/ipykernel_57370/867120382.py:18: RuntimeWarning: The events passed to the Epochs constructor are not chronologically ordered.
  all_epochs = mne.read_epochs(all_epochs_fname)
/var/folders/hp/k3clndsn2sj608qsb4fl5zvr0000gn/T/ipykernel_57370/867120382.py:21: RuntimeWarning: This filename (/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/MEG_Epochs/theta/Patient5_part1L_high_theta_epochs.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  high_epochs = mne.read_epochs(high_epochs_fname)


Not setting metadata
30 matching events found
No baseline correction applied
0 projection items activated
Reading /Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/MEG_Epochs/theta/Patient5_part1L_low_theta_epochs.fif ...
    Found the data of interest:
        t =   -3000.00 ...    3000.00 ms
        0 CTF compensation matrices available


...

Setting up band-pass filter from 2 - 5 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 2.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 1.00 Hz)
- Upper passband edge: 5.00 Hz
- Upper transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 6.00 Hz)
- Filter length: 1651 samples (1.651 s)



[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  71 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done 161 tasks      | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done 287 tasks      | elapsed:    0.2s
[Parallel(n_jobs=1)]: Done 449 tasks      | elapsed:    0.3s
[Parallel(n_jobs=1)]: Done 647 tasks      | elapsed:    0.4s
[Parallel(n_jobs=1)]: Done 881 tasks      | elapsed:    0.5s
[Parallel(n_jobs=1)]: Done 1151 tasks      | elapsed:    0.5s
[Parallel(n_jobs=1)]: Done 1457 tasks      | elapsed:    0.6s
[Parallel(n_jobs=1)]: Done 1799 tasks      | elapsed:    0.7s
[Parallel(n_jobs=1)]: Done 2177 tasks      | elapsed:    0.8s
[Parallel(n_jobs=1)]: Done 2591 tasks      | elapsed:    0.9s
[Parallel(n_jobs=1)]: Done 3041 tasks      | elapsed:    1.2s
[Parallel(n_jobs=1)]: Done 3527 tasks      | elapsed:    1.4s
[Parallel(n_jobs=1)]: Done 4049 tasks      | elapsed:    1.6s
[Parallel(n_jobs=1)]: Done 4607 tasks      | elapsed:    2.1s
[Parallel(n_job

Boundary surface file : /Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/freesurfer/recons/patient_25/bem/inner_skull.surf
grid                  : 10.0 mm
mindist               : 5.0 mm
MRI volume            : /Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/freesurfer/recons/patient_25/mri/T1.mgz

Reading /Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/freesurfer/recons/patient_25/mri/T1.mgz...

Loaded bounding surface from /Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/freesurfer/recons/patient_25/bem/inner_skull.surf (10242 nodes)
Surface CM = ( -16.7   -1.5  -46.5) mm
Surface fits inside a sphere with radius  112.5 mm
Surface extent:
    x =  -97.5 ...   72.3 mm
    y =  -85.8 ...   79.8 mm
    z = -126.4 ...   56.7 mm
Grid extent:
    x = -100.0 ...   80.0 mm
    y =  -90.0 ...   80.0 mm
    z = -130.0 ...   60.0 mm
6840 sources before omitting any.
5214 sources after omitting infeasible sources not within 0.0 - 112.5 mm.
Source spaces are in MRI coordinates.
Checking that the sources are inside the surface a

[Parallel(n_jobs=1)]: Done 8711 tasks      | elapsed:    4.7s


Checking surface interior status for 5214 points...
    Found  141/5214 points inside  an interior sphere of radius   32.4 mm
    Found    0/5214 points outside an exterior sphere of radius  112.5 mm
    Found 2689/5073 points outside using surface Qhull
    Found  699/2384 points outside using solid angles
    Total 1826/5214 points inside the surface
Interior check completed in 3423.0 ms
    3388 source space points omitted because they are outside the inner skull surface.
    332 source space points omitted because of the    5.0-mm distance limit.
1494 sources remaining after excluding the sources outside the surface and less than    5.0 mm inside.
Adjusting the neighborhood info.
Source space : MRI voxel -> MRI (surface RAS)
    0.010000 0.000000 0.000000    -100.00 mm
    0.000000 0.010000 0.000000     -90.00 mm
    0.000000 0.000000 0.010000    -130.00 mm
    0.000000 0.000000 0.000000       1.00
MRI volume : MRI voxel -> MRI (surface RAS)
    -0.001000 0.000000 0.000000     128.